In [8]:
import pandas as pd
import numpy as np
import networkx as nx

import matplotlib.pyplot as plt
import math as m

from utility import log

from networkx.algorithms import approximation
from collections import Counter


from scipy.sparse import csr_matrix



# Preprocessing
Before starting with the actual algorithms we need to apply some prepocessing. For example removing data that is not relevant for our research, and extracting all nodes and links.

In [2]:
# Load the datafile
log("Loading data")
file_path = "data/soc-redditHyperlinks-body.tsv"
raw_data = pd.read_csv(file_path, sep="\t") 

[12:03:32] Loading data


In [3]:
# Remove columns we won't be using
log("Dropping")
data = raw_data.drop(columns=['POST_ID', 'PROPERTIES'])

# Create a list of all nodes
log("Creating list of all nodes")
nodes = pd.concat([data['SOURCE_SUBREDDIT'], data['TARGET_SUBREDDIT']], axis=0)
nodes.drop_duplicates(inplace=True)


log(f"Nodes: {nodes.size:,}")
log(f"Links: {data.shape[0]:,}")

[12:03:37] Dropping
[12:03:37] Creating list of all nodes
[12:03:37] Nodes: 35,776
[12:03:37] Links: 286,561


In [4]:
# Reduce the amount of links to just one direct link.
# This can be useful if we want a more compressed view of the data.
links_compressed = data.drop(columns=["TIMESTAMP"]).groupby(['SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT']).agg('sum')

log(f"Compressed links: {links_compressed.shape[0]:,}")

[12:03:39] Compressed links: 137,821


In [5]:
log("Preparing nx.MultiDiGraph()...")
mGraph = nx.MultiDiGraph()

log("Adding nodes...")
for node in nodes:
    mGraph.add_node(node)

log("Adding edges...")
for index, edge in data.iterrows():
    mGraph.add_edge(edge['SOURCE_SUBREDDIT'], edge['TARGET_SUBREDDIT'], sentiment=edge['LINK_SENTIMENT'])

[12:03:41] Preparing nx.MultiDiGraph()...
[12:03:41] Adding nodes...
[12:03:42] Adding edges...


# Anaylsis
This is the analysis of the network

In [6]:
log(f"mGraph > Vertices: \t{mGraph.number_of_nodes():,}")
log(f"mGraph > Edges: \t{mGraph.number_of_edges():,}")

in_degree = pd.DataFrame(mGraph.in_degree(), columns=['SOURCE', 'degree'])
out_degree = pd.DataFrame(mGraph.out_degree(), columns=['SOURCE', 'degree'])

in_degree.sort_values('degree', ascending=False, inplace=True)
out_degree.sort_values('degree', ascending=False, inplace=True)

log("In degree: (Most referenced subreddits)")
print(in_degree.head(5))

log("Out degree: (Referencing other subreddits)")
print(out_degree.head(5))


[12:05:22] mGraph > Vertices: 	35,776
[12:05:22] mGraph > Edges: 	286,561
[12:05:22] In degree: (Most referenced subreddits)
               SOURCE  degree
34          askreddit    7329
22               iama    3694
7741             pics    2779
42     writingprompts    2490
11294          videos    2446
[12:05:22] Out degree: (Referencing other subreddits)
               SOURCE  degree
70     subredditdrama    4665
16        circlebroke    2358
6400  shitliberalssay    1968
126      outoftheloop    1958
79          copypasta    1824


Calculation of centrality indices: Degree Centrality, Betweenness Centrality, Closeness Centrality and Eigenvector Centrality

In [10]:
# Calculate (In/Out)-Degree centrality (in and out degree)
total_degree = in_degree.merge(out_degree, on='SOURCE', suffixes=('_in', '_out'))
total_degree['total_degree_centrality'] = total_degree['degree_in'] + total_degree['degree_out']
total_degree_centrality = total_degree.sort_values('total_degree_centrality', ascending=False)


log("Total Degree-centrality:")
print(total_degree.head(5))


# Prune low connectivity nodes to speed up the computation of certain properties of the graph
def prune_nodes(G):
    threshold = 300
    graph_copy = G.copy()
    nodes_to_remove = total_degree_centrality[total_degree_centrality['total_degree_centrality'] < threshold]['SOURCE'].tolist()
    graph_copy.remove_nodes_from(nodes_to_remove)
    total_nodes = graph_copy.number_of_nodes()
    total_edges = graph_copy.number_of_edges()
    print(f"Total Nodes after Pruning: {total_nodes}")
    print(f"Total Edges after Pruning: {total_edges}")
    return graph_copy

# Calculate betweenness centrality using an approximation measure for time reasons
# If we calculate betweenness centrality for k=n then it will take too much (we can use a sample of k nodes)

betweennes_calculation = nx.betweenness_centrality(mGraph, k = 100)
betweenness_centrality = pd.DataFrame(betweennes_calculation.items(), columns=['SOURCE', 'betweenness_centrality'])

betweenness_centrality = betweenness_centrality.sort_values('betweenness_centrality', ascending=False)

log("Betweenness Centrality: (Influence of node in graph)")
print(betweenness_centrality.head(5))


# Calculate eigenvector centrality. We need a function
# which does that for multiGraph instead of singleGraph
# Used the same start parameters (mat_iter and tol) as NetworkX
def m_eigenvector_centrality(G, max_iter=100, tol=1e-6):
    # Prune some nodes of the matrix otherwise my PC crashes!
    G_copy = prune_nodes(G)
    # Make an adjacency matrix
    adjacency_matrix = csr_matrix(nx.adjacency_matrix(G_copy))

    # Array to save eigenvector centrality for each node
    num_nodes = len(G_copy.nodes())
    eigenvector = np.full(num_nodes, 1.0 / num_nodes)

    for _ in range(max_iter):
        # Calculate the matrix-vector product
        new_eigenvector = np.dot(adjacency_matrix.toarray(), eigenvector)

        # Normalize
        norm = np.linalg.norm(new_eigenvector)
        new_eigenvector = new_eigenvector / norm

        # Repeat until convergence (or max_iter)
        if np.linalg.norm(new_eigenvector - eigenvector) < tol:
            return dict(zip(G_copy.nodes(), new_eigenvector))

        eigenvector = new_eigenvector

    return dict(zip(G_copy.nodes(), eigenvector))

eigenvector_calculation = m_eigenvector_centrality(mGraph)
eigenvector_centrality = pd.DataFrame(eigenvector_calculation.items(), columns=['SOURCE', 'eigenvector_centrality'])
eigenvector_centrality = eigenvector_centrality.sort_values('eigenvector_centrality', ascending=False)

log("Top 5 Nodes by Eigenvector Centrality:")
print(eigenvector_centrality.head(5))

# Use a subset of nodes to calculate closeness centrality for. The subset is chosen as
# the combination of other centrality values
combined_centrality = total_degree_centrality.merge(betweenness_centrality, on='SOURCE')
combined_centrality = combined_centrality.merge(eigenvector_centrality, on='SOURCE')

# Just sum the values
combined_centrality['combined_centrality'] = (
    combined_centrality['total_degree_centrality'] +
    combined_centrality['betweenness_centrality'] +
    combined_centrality['eigenvector_centrality']
)

combined_centrality = combined_centrality.sort_values('combined_centrality', ascending=False)
# Select the top 20
top_nodes = combined_centrality.head(20)
top_nodes_list = top_nodes['SOURCE'].tolist()

closeness_centrality_calc = {}

for node in top_nodes_list:
    closeness_centrality_calc[node] = nx.closeness_centrality(mGraph, u=node)
closeness_centrality = pd.DataFrame(closeness_centrality_calc.items(), columns=['SOURCE', 'closeness_centrality'])
closeness_centrality = closeness_centrality.sort_values('closeness_centrality', ascending=False)

log("Top 5 Nodes by Closeness Centrality:")
print(closeness_centrality.head(5))

[12:08:35] Total Degree-centrality:
           SOURCE  degree_in  degree_out  total_degree_centrality
0       askreddit       7329        1338                     8667
1            iama       3694        1181                     4875
2            pics       2779           4                     2783
3  writingprompts       2490        1707                     4197
4          videos       2446           4                     2450
[12:08:45] Betweenness Centrality: (Influence of node in graph)
             SOURCE  betweenness_centrality
22             iama                0.050840
34        askreddit                0.046842
70   subredditdrama                0.046078
126    outoftheloop                0.042275
42   writingprompts                0.019658
Total Nodes after Pruning: 314
Total Edges after Pruning: 56691
[12:08:46] Top 5 Nodes by Eigenvector Centrality:
             SOURCE  eigenvector_centrality
51         buildapc                0.690689
131     techsupport                0.6

Calculation of Clustering Coefficient

In [29]:
# As before, we need to make our own function since
# networkX does not work for MultiDiGraphs
def f_clustering_coefficient(G, node):
    # Get the neighbors of the given node
    neighbors = set(G.neighbors(node))

    # If there are no neighbors then return 0
    if len(neighbors) < 2:
        return 0.0

    # Find the number of edges between neighbors
    edges_between_neighbors = 0

    for u in neighbors:
        for v in neighbors:
            if u != v and G.has_edge(u, v):
                # Usually edge weights are not considered, hence we just add +1
                edges_between_neighbors += 1

    # Calculate the clustering coefficient
    clustering_coefficient = edges_between_neighbors / (len(neighbors) * (len(neighbors) - 1))
    return clustering_coefficient


clustering_coefficient_calc = {}
for node in mGraph.nodes():
    clustering_coefficient_calc[node] = f_clustering_coefficient(mGraph, node)

clustering_coefficient = pd.DataFrame(clustering_coefficient_calc.items(), columns=['SOURCE', 'clustering_coefficient'])
count_of_ones = len(clustering_coefficient[clustering_coefficient['clustering_coefficient'] == 1])

# 1 is the highest clustering coefficient value
log("Number of nodes with a clustering coefficient = 1:")
print(count_of_ones)


[12:01:43] Number of nodes with a clustering coefficient = 1:
1498


Network Diameter + Number of Connected Components + Size of connected components + Density

In [12]:
strongly_connected_components = list(nx.strongly_connected_components(mGraph))
component_sizes = [len(component) for component in strongly_connected_components]
sorted_sizes = sorted(component_sizes, reverse=True)

log("Number of Strongly Connected Components:")
total_Scc = len(strongly_connected_components)
print(total_Scc)
log("Top 5 Strongly Connected Components by size:")
for i, size in enumerate(sorted_sizes[:5], start=1):
    print(f"Component {i}: Size = {size} nodes")

density = nx.density(mGraph)
log("The density of the Graph is")
print(density)

# Calculate diameter only for the biggest SCC
largest_scc = max(strongly_connected_components, key=len)
largest_scc_subgraph = mGraph.subgraph(largest_scc)
# diameter = nx.diameter(largest_scc_subgraph, weight="weight", usebounds=True)
diameter = approximation.diameter(largest_scc_subgraph)

log("Diameter approximation of largest SCC:")
print(diameter)

[11:07:58] Number of Strongly Connected Components:
24071
[11:07:58] Top 5 Strongly Connected Components by size:
Component 1: Size = 11564 nodes
Component 2: Size = 6 nodes
Component 3: Size = 5 nodes
Component 4: Size = 4 nodes
Component 5: Size = 4 nodes
[11:07:58] The density of the Graph is
0.00022389565198911403
[11:08:02] Diameter approximation of largest SCC:
12


Find Cliques 

In [10]:
# Convert to undirected graph. Explanation to why this is fine (https://stackoverflow.com/questions/12896477/cliques-for-directed-graphs-in-igraph)
undirected_graph = nx.Graph()

# Add an edge (u,v) in the undirected graph iff in our multidigraph we have
# (u,v) and (v,u)
for u, v in mGraph.edges():
    # Check if there is an edge (v, u) in the MultiDiGraph
    if mGraph.has_edge(v, u):
        undirected_graph.add_edge(u, v)

cliques = list(nx.find_cliques(undirected_graph))

sorted_cliques = sorted(cliques, key=lambda x: len(x), reverse=True)

# Print the 5 largest cliques
for i, clique in enumerate(sorted_cliques[:5], start=1):
    print(f"Largest Clique {i}, Length {len(clique)}: {clique}")

Largest Clique 1, Length 33: ['greenbaypackers', 'seahawks', '49ers', 'ravens', 'azcardinals', 'nfl', 'panthers', 'oaklandraiders', 'texans', 'bengals', 'colts', 'steelers', 'eagles', 'buffalobills', 'chibears', 'nfl_draft', 'falcons', 'browns', 'denverbroncos', 'cowboys', 'kansascitychiefs', 'saints', 'jaguars', 'nyjets', 'chargers', 'redskins', 'detroitlions', 'buccaneers', 'miamidolphins', 'nygiants', 'minnesotavikings', 'losangelesrams', 'patriots']
Largest Clique 2, Length 33: ['greenbaypackers', 'seahawks', '49ers', 'ravens', 'azcardinals', 'nfl', 'panthers', 'oaklandraiders', 'texans', 'bengals', 'colts', 'steelers', 'eagles', 'buffalobills', 'chibears', 'nfl_draft', 'falcons', 'browns', 'denverbroncos', 'cowboys', 'kansascitychiefs', 'saints', 'jaguars', 'nyjets', 'chargers', 'redskins', 'detroitlions', 'buccaneers', 'miamidolphins', 'nygiants', 'minnesotavikings', 'losangelesrams', 'tennesseetitans']
Largest Clique 3, Length 10: ['mbotandgtron', 'nomina', 'cqb', 'pears', 'kill

Run homophily test with NSFW and SFW subreddits (not done yet)

Important nodes acting as Bridges. Calculation for MultiDiGraph (Paper: https://core.ac.uk/reader/82453490) 

In [27]:
# We say that a bridge in a Graph is a strong bridge if removing it
# increases the number of SCC (ref: paper above)

def find_bridges(G):
    bridges = []

    total_scc = len(list(nx.strongly_connected_components(G)))
    graph_copy = G.copy()
    count = 0
    for u, v in G.edges():
        if count % 5000 == 0:
            print(f"Processed {count} edges")
        count += 1
        # Temporarily remove the edge (u, v)
        graph_copy.remove_edge(u, v)
        new_scc_count = len(list(nx.strongly_connected_components(graph_copy)))

        # If the number of new SCC < original SCC then the edge is bridge
        if new_scc_count > total_scc:
            bridges.append((u, v))

        # Restore edge (u, v)
        graph_copy.add_edge(u, v)
    return bridges

# Find bridge edges in the undirected graph
bridge_edges = list(find_bridges(prune_nodes(mGraph)))
print(bridge_edges)
bridge_nodes = [node for edge in bridge_edges for node in edge]

# Count the occurrences of each node
top_nodes = Counter(bridge_nodes)

# Get the top 5 nodes with the highest counts
top_nodes = top_nodes.most_common(5)

# Print the top 5 nodes
print("Top 5 Nodes in Bridge Edges:")
for node, count in top_nodes:
    print(f"Node: {node}, Count: {count}")

Total Nodes after Pruning: 314
Total Edges after Pruning: 56691
Processed 0 edges
Processed 5000 edges
Processed 10000 edges
Processed 15000 edges
Processed 20000 edges
Processed 25000 edges
Processed 30000 edges
Processed 35000 edges
Processed 40000 edges
Processed 45000 edges
Processed 50000 edges
Processed 55000 edges
[('footballhighlights', 'soccer'), ('gifs', 'resissues'), ('pics', 'changelog'), ('ironthronepowers', 'bugs'), ('modelusgov', 'mhoc')]
Top 5 Nodes in Bridge Edges:
Node: footballhighlights, Count: 1
Node: soccer, Count: 1
Node: gifs, Count: 1
Node: resissues, Count: 1
Node: pics, Count: 1


Important nodes acting as Bridges. Calculation for Undirected graph

In [28]:
simple_graph = mGraph.to_undirected()

# Find bridge edges in the undirected graph
bridge_edges = list(nx.bridges(simple_graph))

bridge_nodes = [node for edge in bridge_edges for node in edge]

# Count the occurrences of each node
top_nodes = Counter(bridge_nodes)

# Get the top 5 nodes with the highest counts
top_nodes = top_nodes.most_common(5)

# Print the top 5 nodes
print("Top 5 Nodes in Bridge Edges (undirected graphs):")
for node, count in top_nodes:
    print(f"Node: {node}, Count: {count}")

Top 5 Nodes in Bridge Edges (undirected graphs):
Node: askreddit, Count: 291
Node: writingprompts, Count: 188
Node: tipofmypenis, Count: 188
Node: iama, Count: 183
Node: mhoc, Count: 133


Implement Girvan-Newman algorithm (not done yet)